In [1]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LassoCV, RidgeCV, Ridge, Lasso

In [2]:
df = pd.read_csv('kaggle/input/2020_hw3_team_7_ds_train_full.csv', index_col='id')
# df, y_train = df.loc[:, df.columns != 'target'], df.target

hold_out = pd.read_csv('kaggle/input/2020_hw3_team_7_ds_test.csv', index_col='id')

In [3]:
df.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,target
id,,,,,,,,,,,,,
0,7.6,0.39,0.22,2.80,0.036,19.0,113.0,0.9926,3.03,0.29,10.2,w,5
1,8.5,0.17,0.49,8.80,0.048,23.0,108.0,0.9947,2.88,0.34,10.5,w,4
2,6.0,0.41,0.21,1.90,0.050,29.0,122.0,0.9928,3.42,0.52,10.5,w,6
3,6.7,0.34,0.30,15.60,0.054,51.0,196.0,0.9982,3.19,0.49,9.3,w,5
4,6.2,0.25,0.21,15.55,0.039,28.0,159.0,0.9982,3.48,0.64,9.6,w,6


In [4]:
df.describe()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,target
count,4547.000000,4547.000000,4547.000000,4547.000000,4547.000000,4547.000000,4547.00000,4547.000000,4547.000000,4547.000000,4547.000000,4547.000000
mean,7.213602,0.338734,0.319463,5.467341,0.056096,30.834506,116.32439,0.994697,3.216050,0.530207,10.487887,5.818782
std,1.288101,0.165001,0.145727,4.815497,0.035586,17.919255,56.74980,0.003040,0.160897,0.148056,1.196249,0.873760
min,3.900000,0.080000,0.000000,0.600000,0.009000,1.000000,6.00000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,78.00000,0.992300,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.290000,0.310000,3.000000,0.048000,29.000000,118.00000,0.994880,3.200000,0.500000,10.300000,6.000000
75%,7.700000,0.400000,0.390000,8.200000,0.065000,42.000000,156.00000,0.997000,3.320000,0.600000,11.300000,6.000000
max,15.600000,1.580000,1.660000,65.800000,0.611000,289.000000,440.00000,1.038980,4.010000,2.000000,14.200000,9.000000


In [5]:
df.f12.unique()

array(['w', 'r'], dtype=object)

In [6]:
# for col in df.columns:
#     print(f'{col:<25} {df[col].unique()}')

In [7]:
df['f12'].replace({'w': 0, 'r': 1}, inplace=True)
hold_out['f12'].replace({'w': 0, 'r': 1}, inplace=True)        

In [8]:
def RMSE(y_train, y_pred):
    mse = mean_squared_error(y_train, y_pred)
    return np.sqrt(mse)

In [9]:
# unused
def pipeline_factory():
    return Pipeline([
        ("scale", StandardScaler()),
        ("select features",VarianceThreshold()),
        ("model", LinearRegression(n_jobs=-1))
    ])

#### Model

In [10]:
scaler = StandardScaler()
model = LinearRegression(n_jobs=-1)

In [11]:
X_train, y_train = df.loc[:, df.columns != 'target'], df.target

### Scaling

In [12]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(hold_out)

In [13]:
model.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

#### За основу беру линейную регрессию, по ней смотрю фичи с помощью permutation_score, потом эти фичи засовываю в KNN

In [14]:
from sklearn.inspection import permutation_importance
r = permutation_importance(model, X_train_scaled, y_train,
                           n_repeats=30,
                           random_state=0)

to_save = []
for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{X_train.columns[i]:<27}"
              f"{r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")
        to_save.append(X_train.columns[i])

f8                         0.285 +/- 0.010
f4                         0.267 +/- 0.009
f11                        0.188 +/- 0.008
f2                         0.168 +/- 0.007
f12                        0.071 +/- 0.004
f1                         0.034 +/- 0.003
f10                        0.032 +/- 0.003
f7                         0.018 +/- 0.002
f9                         0.015 +/- 0.002
f6                         0.014 +/- 0.002
f5                         0.001 +/- 0.000


#### если выбрать все фичи (без выборки to_save), то результат совсем мало отличается, но будет чуть хуже


In [15]:
X_train_scaled = scaler.fit_transform(X_train[to_save])
X_test_scaled = scaler.transform(hold_out[to_save])

var = VarianceThreshold(0.1)
X_train_scaled = var.fit_transform(X_train_scaled)
X_test_scaled = var.transform(X_test_scaled)

In [16]:
from sklearn.model_selection import KFold

def estimate_model(X, y):
    k_fold = KFold(n_splits=10, shuffle=False)
    total_scores = {}
    
    pipes = create_pipelines(1, 21)
    for k, pipe in pipes.items():
        k_scores = []
        for train_index, test_index in k_fold.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            # Train
            pipe.fit(X_train, y_train)

            # Test
            y_pred = pipe.predict(X_test)
            score = accuracy_score(y_test, y_pred)
            k_scores.append(score)
        total_scores[pipe] = (np.mean(k_scores), np.std(k_scores))
    return total_scores

def create_pipelines(lower_bound, upper_bound):
    k_range = range(lower_bound, upper_bound)
    pipelines = {}
    for k in k_range:    
        pipelines[k] = Pipeline([
                                ("scale", StandardScaler()),
                                ("var", VarianceThreshold(0.1)), # не было до этого
                                ("clf", KNeighborsClassifier(n_neighbors=k))
                                ])
    return pipelines

In [17]:
scores = estimate_model(X_train_scaled, y_train)

In [18]:
model = [(k,v) for k, v in sorted(scores.items(), key=lambda item: item[1])][-1][0]

In [19]:
preds = model.predict(X_test_scaled)
subm = pd.DataFrame({"id":range(X_test_scaled.shape[0]), 'target':preds})
subm.to_csv('submission.csv', index=False)

In [20]:
np.unique(y_train)

array([3, 4, 5, 6, 7, 8, 9])

In [21]:
np.unique(preds)

array([3, 4, 5, 6, 7, 8, 9])

In [22]:
!cat submission.csv

id,target
0,6
1,6
2,5
3,5
4,5
5,6
6,7
7,6
8,6
9,7
10,6
11,6
12,7
13,6
14,6
15,7
16,6
17,5
18,5
19,6
20,7
21,5
22,6
23,7
24,6
25,7
26,6
27,6
28,5
29,6
30,5
31,6
32,6
33,5
34,6
35,4
36,7
37,7
38,5
39,6
40,5
41,7
42,7
43,6
44,5
45,7
46,5
47,5
48,6
49,7
50,6
51,5
52,5
53,6
54,6
55,4
56,6
57,6
58,7
59,7
60,6
61,5
62,8
63,6
64,7
65,6
66,6
67,6
68,6
69,6
70,7
71,5
72,7
73,6
74,6
75,6
76,5
77,5
78,5
79,6
80,7
81,7
82,7
83,7
84,6
85,6
86,6
87,7
88,7
89,6
90,8
91,6
92,5
93,5
94,7
95,5
96,6
97,6
98,6
99,5
100,7
101,6
102,6
103,5
104,6
105,5
106,4
107,6
108,5
109,7
110,6
111,8
112,7
113,6
114,5
115,6
116,5
117,6
118,6
119,6
120,5
121,5
122,6
123,6
124,5
125,6
126,5
127,5
128,5
129,6
130,7
131,4
132,6
133,6
134,6
135,6
136,5
137,5
138,5
139,6
140,5
141,7
142,5
143,5
144,5
145,7
146,5
147,6
148,7
149,6
150,6
151,6
152,8
153,6
154,6
155,7
156,6
157,6
158,6
159,5
160,5
161,5
162,6
163,7
164,5
165,5
166,6
167,5
168,7
169,5
170,7
171,5
172,6
173,6
174,6
175,6
176,5
177,6
178,5
179,5
180,6
181,6
182,7
18

https://www.kaggle.com/c/csc-hw3-autumn2020-team-7/overview